<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: Reza Vahidimajd <br>
   **Student ID**: 99102491 <br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split

print("in this data set, the first column is number of attended courses and second column is study time and in the third \n" + 
      "column we have gained marks based of the first 2 columns, so in this model we want to reach hyperparameters\n" + 
     "to estimate marks based on the given attended courses and study times.\n \n")

np.set_printoptions(formatter={'float_kind':'{:f}'.format})
data = pd.read_csv('Student_Marks.csv', header = None)
data = np.genfromtxt('Student_Marks.csv', delimiter=',')

train_data = data
#test_data = data[int(len(data) * 7/10):]
#append a column with 1 to the start of the X

X = train_data[:, :-1]
X = np.append(np.ones((len(X), 1)), X , axis=1)
Y = train_data[:, -1]



def lr_predict(X, W):
    return np.dot(X, W)

def calc_error(predictions, Y):
    return predictions - Y

def mse_loss(predictions, Y):
    difference = predictions - Y
    loss = np.sum(difference * difference) / difference.size
    return loss

def calc_gradient(X, error):
    return np.dot(X.T, error)

def update_weights(W, learning_rate , gradient):
    return W - learning_rate * gradient

def train_model(X, W, Y, learning_rate):
    
    losses = []
    
    for i in range(100):
        predictions = lr_predict(X, W)
        losses.append(mse_loss(predictions, Y))
        gradient = calc_gradient(X, calc_error(predictions, Y)) / Y.size
        W = update_weights(W, learning_rate, gradient)
        
    return losses[-1]    

losses = []
x = []


for i in range(100, 1, -1):
    
    W = np.random.rand(3, 1)
    x.append(i / 100000)
    losses.append(train_model(X, W, Y, i / 100000))


print("best hyperparameter: " + str(x[losses.index(min(losses))]) + "\n")




#use calculated hyperparameters in the last section and X of the test data to find estimated Ys.
#then compare estimated with real Y by using mean squared error

#print(pd.DataFrame(result))


#lasso regression

x_train, x_test, y_train, y_test = train_test_split(train_data[:, :-1], train_data[:, -1], test_size = 0.3)

losses = []
a = []
y_lasso = []

for i in range(1, 100):
    
    lasso_reg = Lasso(alpha = i/100)

    lasso_reg.fit(x_train, y_train)

    Y_prediction_lass =lasso_reg.predict(x_test)
    y_lasso.append(Y_prediction_lass)
    losses.append(mse_loss(Y_prediction_lass, y_test))
    a.append(i/100)

best_lasso = y_lasso[losses.index(min(losses))]
print("lowest loss (on test) lasso: " + str(min(losses)))
print("best alpha lasso: " + str(a[losses.index(min(losses))]) + "\n")

#ridge regression

losses = []
a = []
y_ridge = []

for i in range(1, 100):
    
    ridge_reg = Ridge(alpha = i/100)

    ridge_reg.fit(x_train, y_train)

    Y_prediction_ridge = ridge_reg.predict(x_test)
    y_ridge.append(Y_prediction_ridge)
    losses.append(mse_loss(Y_prediction_ridge, y_test))
    a.append(i/100)

best_ridge = y_ridge[losses.index(min(losses))]
print("lowest loss ridge (on test): " + str(min(losses)))
print("best alpha ridge: " + str(a[losses.index(min(losses))]) + "\n")

#Elastic Net


losses = []
a = []
y_elastic = []

for i in range(1, 100):
    for j in range(1, 100):
        elastic_reg = ElasticNet(alpha = i/100, l1_ratio = j / 100)
        a.append([i/100, j/100])
        elastic_reg.fit(x_train, y_train)
        
        Y_prediction_elastic = elastic_reg.predict(x_test)
        y_elastic.append(Y_prediction_elastic)
        losses.append(mse_loss(Y_prediction_elastic, y_test))

best_elastic = y_elastic[losses.index(min(losses))]
print("lowest loss ridge (on test): " + str(min(losses)))
print("best alpha and l1_ration ElasticNet: " + str(a[losses.index(min(losses))]) + "\n")
#compare 

result = {'real value':  y_test
          , 'lasso': best_lasso
          , 'ridge ': best_ridge
          , 'elastic net': best_elastic
        }

print(pd.DataFrame(result))

in this data set, the first column is number of attended courses and second column is study time and in the third 
column we have gained marks based of the first 2 columns, so in this model we want to reach hyperparameters
to estimate marks based on the given attended courses and study times.
 

best hyperparameter: 0.00074

lowest loss (on test) lasso: 11.451467472205765
best alpha lasso: 0.72

lowest loss ridge (on test): 11.658354898912927
best alpha ridge: 0.99

lowest loss ridge (on test): 11.396157667443362
best alpha and l1_ration ElasticNet: [0.23, 0.01]

    real value      lasso     ridge   elastic net
0       16.606  20.531386  20.221458    20.440069
1       38.490  39.904819  40.581069    40.021977
2       19.128  23.001350  23.059243    23.140620
3       19.466  23.338794  23.077597    23.198325
4       35.939  36.743718  37.040151    36.642390
5       11.397  14.391477  13.812499    14.270741
6       18.238  21.233370  21.423049    21.540509
7        6.349   3.124342   2.